In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
#判断cuda是否可用，可用就在gpu上运行，不可用就在cpu上
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):   ##父类
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()   ##创建flatten
        self.linear_relu_stack = nn.Sequential(  ##线性+relu的堆叠模块
            nn.Linear(28*28, 512),  ##线性层  Linear（输入特征的特征维度，隐含层的大小）
            nn.ReLU(),              ##relu非线性的激活函数
            nn.Linear(512, 512),    ##输入为512，输出也为512
            nn.ReLU(),
            nn.Linear(512, 10),     ##输入为512，输出为10
        )                           ##一共五层

    def forward(self, x):           ##默认x作为输入  前向运算
        x = self.flatten(x)         ##维度摊平
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)  
print(model)   ##print输出

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
#如何在jupyter notebook中安装github中的第三方库
from torchsummary import summary
summary(model, input_size=(1, 28, 28)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [7]:
X = torch.rand(1, 28, 28, device=device)  ##在device随机初始化一个1*28*28的张量
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)   ##实例化softmax
y_pred = pred_probab.argmax(1)            ##计算出概率最大值
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())  ##查看输入张量的大小

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()     ##实例化操作，flatten的作用是从第一维度到最后一个维度浓缩成一个维度
flat_image = flatten(input_image)
print(flat_image.size())  ##输出的784=28*28

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)   ## Linear（输入的特征大小，输出的隐含层大小）
hidden1 = layer1(flat_image)
print(hidden1.size())      ##784=28*28被out_features=20替换

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1900, -0.3520,  0.4229, -0.0819,  0.0373,  0.0321,  0.6072,  0.0025,
          0.0547, -0.3817,  0.3305, -0.1969, -0.0204,  0.2798, -0.0830, -0.1471,
         -0.3000, -0.1589, -0.1237,  0.4137],
        [ 0.2337, -0.3599,  0.2771, -0.1569,  0.2875, -0.4632,  0.7306,  0.2544,
         -0.0988, -0.3940,  0.0674, -0.1030,  0.6177,  0.2127,  0.0433, -0.1029,
         -0.4432, -0.2290, -0.1548,  0.3283],
        [ 0.2453, -0.3998,  0.2494, -0.1623, -0.1197, -0.2783,  0.2950,  0.3928,
         -0.0547, -0.1786,  0.1675, -0.1743,  0.4023, -0.0279,  0.0020,  0.0102,
         -0.3827,  0.0755, -0.0959,  0.2682]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1900, 0.0000, 0.4229, 0.0000, 0.0373, 0.0321, 0.6072, 0.0025, 0.0547,
         0.0000, 0.3305, 0.0000, 0.0000, 0.2798, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.4137],
        [0.2337, 0.0000, 0.2771, 0.0000, 0.2875, 0.0000, 0.7306, 0.2544, 0.0000,
         0.0000, 0.0674, 0.0000, 0.6177, 0.2127, 0.04

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0027, -0.0300, -0.0350,  ...,  0.0225,  0.0208, -0.0184],
        [-0.0221, -0.0243,  0.0132,  ..., -0.0014,  0.0102,  0.0226]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0155, -0.0339], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0143,  0.0362,  0.0058,  ...,  0.0147, -0.0175, -0.0181],
        [ 0.0117, -0.0362, -0.0252,  ..., -0.0401, -0.0408, -0.0132]],
       device='cuda:0', grad_fn=<Sl